<a href="https://colab.research.google.com/github/ruchi0451/NLP_AD/blob/main/GAN_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from IPython import get_ipython
from IPython.display import display
# %%
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from keras.layers import Input, Dense, Dropout, LeakyReLU
from keras.models import Model, Sequential
from keras.datasets import mnist
from keras.optimizers import Adam
from keras import initializers
import tensorflow as tf # Import TensorFlow

# Enforce eager execution
tf.config.run_functions_eagerly(True) # Enable eager execution

# %%
np.random.seed(10)
random_dim = 100
# %%
def load_minst_data():
    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    x_train = (x_train.astype(np.float32) - 127.5)/127.5
    x_train = x_train.reshape(60000, 784)
    return (x_train, y_train, x_test, y_test)

# %%
def get_optimizer():
  return Adam(learning_rate=0.0002, beta_1=0.5)
# %%
def get_generator(optimizer):
  generator = Sequential()
  generator.add(Dense(256, input_dim=random_dim, kernel_initializer=initializers.RandomNormal(stddev=0.02)))
  generator.add(LeakyReLU(0.2))
  generator.add(Dense(512))
  generator.add(LeakyReLU(0.2 ))
  generator.add(Dense(1024))
  generator.add(LeakyReLU(0.2))
  generator.add(Dense(784, activation='tanh'))
  generator.compile(loss='binary_crossentropy', optimizer=optimizer)
  return generator
# %%
def get_discriminator(optimizer):
  discriminator = Sequential()

  discriminator.add(Dense(256, input_dim=784, kernel_initializer=initializers.RandomNormal(stddev=0.02))) # Changed input_dim to 784
  discriminator.add(LeakyReLU(0.2))
  discriminator.add(Dropout(0.3))
  discriminator.add(Dense(512))
  discriminator.add(LeakyReLU(0.2 ))
  discriminator.add(Dropout(0.3))
  discriminator.add(Dense(256))
  discriminator.add(LeakyReLU(0.2))
  discriminator.add(Dropout(0.3))
  discriminator.add(Dense(784, activation='sigmoid')) # This output shape can be adjusted based on your desired output
  discriminator.compile(loss='binary_crossentropy', optimizer=optimizer)
  return discriminator
# %%
def get_discriminator(optimizer):
  discriminator = Sequential()
  # The input_dim should match the output shape of the generator
  discriminator.add(Dense(256, input_dim=784, kernel_initializer=initializers.RandomNormal(stddev=0.02))) # Changed input_dim to 784
  discriminator.add(LeakyReLU(0.2))
  discriminator.add(Dropout(0.3))
  discriminator.add(Dense(512))
  discriminator.add(LeakyReLU(0.2 ))
  discriminator.add(Dropout(0.3))
  discriminator.add(Dense(256))
  discriminator.add(LeakyReLU(0.2))
  discriminator.add(Dropout(0.3))
  # Change the output layer to have a single neuron with sigmoid activation
  discriminator.add(Dense(1, activation='sigmoid'))  # Output a single probability
  discriminator.compile(loss='binary_crossentropy', optimizer=optimizer)
  return discriminator
# %%
def get_gan_network(discriminator, random_dim, generator, optimizer):
  discriminator.trainable = False
  gan_input = Input(shape=(random_dim,))
  x = generator(gan_input)
  gan_output = discriminator(x)
  gan = Model(inputs=gan_input, outputs=gan_output)
  gan.compile(loss='binary_crossentropy', optimizer=optimizer)
  return gan
# %%
def plot_generated_images(epoch, generator, examples=100, dim=(10, 10), figsize=(10, 10)):
  noise = np.random.normal(0, 1, size=[examples, random_dim])
  generated_images = generator.predict(noise)
  generated_images = generated_images.reshape(examples, 28, 28)
  plt.figure(figsize=figsize)
  for i in range(generated_images.shape[0]):
    plt.subplot(dim[0], dim[1], i+1)
    plt.imshow(generated_images[i], interpolation='nearest', cmap='gray_r')
    plt.axis('off')
    plt.tight_layout()
  plt.savefig('gan_generated_image_epoch_%d.png' % epoch)
# %%
def train(epochs=1, batch_size=128):
  x_train, y_train, x_test, y_test = load_minst_data()
  batch_count = x_train.shape[0] / batch_size
  adam = get_optimizer()
  generator = get_generator(adam)
  discriminator = get_discriminator(adam)
  gan = get_gan_network(discriminator, random_dim, generator, adam)

  # Create a new optimizer instance for the GAN
  gan_optimizer = get_optimizer()
  gan.compile(loss='binary_crossentropy', optimizer=gan_optimizer)  # Re-compile GAN with the new optimizer

  for e in range(1, epochs+1):
    print('-'*15, 'Epoch %d' % e, '-'*15)
    for _ in tqdm(range(batch_size)):
      # ... (rest of your training loop code remains the same)
      noise = np.random.normal(0, 1, size=[batch_size, random_dim])
      image_batch = x_train[np.random.randint(0, x_train.shape[0], size=batch_size)]
      generated_images = generator.predict(noise)
      x = np.concatenate([image_batch, generated_images])
      y_dis = np.zeros(2*batch_size)
      y_dis[:batch_size] = 0.9
      discriminator.trainable = True
      discriminator.train_on_batch(x, y_dis)
      noise = np.random.normal(0, 1, size=[batch_size, random_dim])
      y_gen = np.ones(batch_size)
      discriminator.trainable = False
      gan.train_on_batch(noise, y_gen)
    if e == 1 or e % 20 == 0:
      plot_generated_images(e, generator)

In [ ]:
train(20, 128)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


--------------- Epoch 1 ---------------


  0%|          | 0/128 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/tensorflow/python/data/ops/structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step


  1%|          | 1/128 [00:00<01:02,  2.04it/s]

...

 29%|██▉       | 37/128 [00:16<00:40,  2.26it/s]

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step


 30%|██▉       | 38/128 [00:17<00:40,  2.23it/s]

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step


 30%|███       | 39/128 [00:17<00:39,  2.24it/s]